In [107]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import os


In [108]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#Documentacao:
#https://www.tensorflow.org/tutorials/keras/regression?hl=pt-br

#https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb?hl=pt-br
#

In [109]:


filename=r'D:\seriestesteapp\super_loja.csv'
df_vendas = pd.read_csv(filename, sep=";", encoding="latin-1")
df_vendas['Data do pedido'] = pd.to_datetime(df_vendas['Data do pedido'], errors='coerce')
df_vendas['Data de envio'] = pd.to_datetime(df_vendas['Data de envio'], errors='coerce')


df_vendas[['Vendas', 'Desconto', 'Lucro']] = df_vendas[[
    'Vendas', 'Desconto', 'Lucro']].applymap(lambda k: float(str(k).replace(",", ".")))


df_vendas.dropna(inplace=True)


label_encoder = LabelEncoder()
categorical_columns = df_vendas.select_dtypes(include=['object']).columns
for col in categorical_columns:
    df_vendas[col] = label_encoder.fit_transform(df_vendas[col])


scaler = MinMaxScaler()
scaler_output = MinMaxScaler()
df_vendas[['Vendas', 'Quantidade', 'Desconto', 'Lucro']] = scaler.fit_transform(
    df_vendas[['Vendas', 'Quantidade', 'Desconto', 'Lucro']])
df_vendas['Vendas'] = scaler_output.fit_transform(df_vendas[['Vendas']])


X = df_vendas.drop(columns=['Vendas', 'Data de envio', 'Data do pedido'])
y = df_vendas['Vendas']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


X_train = np.reshape(X_train.values, (X_train.shape[0], X_train.shape[1], 1)).astype(float)
X_test = np.reshape(X_test.values, (X_test.shape[0], X_test.shape[1], 1)).astype(float)


model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(units=100, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(tf.keras.layers.LSTM(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=100, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='linear'))


model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])


epochs_hist = model.fit(X_train, y_train, batch_size=50, epochs=100, validation_split=0.2)


X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
mse, _ = model.evaluate(X_test, y_test)
print(f"Erro Quadrático Médio (MSE) no Conjunto de Teste: {mse:.6f}")

y_pred_scaled = model.predict(X_test)


y_pred_desnormalized = scaler_output.inverse_transform(y_pred_scaled)



min_len = min(len(y_pred_desnormalized), len(df_vendas))
dates_array = df_vendas['Data do pedido'].values
data_envio_array = df_vendas['Data de envio'].values

for i in range(min_len):
    data_pedido = dates_array[i]
    data_envio = data_envio_array[i]
    real = round(scaler_output.inverse_transform(np.array([[df_vendas['Vendas'].iloc[i]]]))[0][0], 6)
    previsto = round(y_pred_desnormalized[i][0], 6)
    print(f"Data do Pedido: {data_pedido}, Data de Envio: {data_envio}, Real: {real}, Previsto: {previsto}")





Epoch 1/100
34/34 [==============================] - 3s 31ms/step - loss: 4320.9868 - mean_squared_error: 4320.9868 - val_loss: 1289.1418 - val_mean_squared_error: 1289.1418
Epoch 2/100
34/34 [==============================] - 1s 23ms/step - loss: 661.4642 - mean_squared_error: 661.4642 - val_loss: 303.1616 - val_mean_squared_error: 303.1616
Epoch 3/100
34/34 [==============================] - 1s 23ms/step - loss: 249.2903 - mean_squared_error: 249.2903 - val_loss: 202.6877 - val_mean_squared_error: 202.6877
Epoch 4/100
34/34 [==============================] - 1s 22ms/step - loss: 106.9727 - mean_squared_error: 106.9727 - val_loss: 87.3396 - val_mean_squared_error: 87.3396
Epoch 5/100
34/34 [==============================] - 1s 21ms/step - loss: 55.0137 - mean_squared_error: 55.0137 - val_loss: 45.7027 - val_mean_squared_error: 45.7027
Epoch 6/100
34/34 [==============================] - 1s 22ms/step - loss: 39.6329 - mean_squared_error: 39.6329 - val_loss: 41.1432 - val_mean_squared_e

In [110]:
y_pred = model.predict(X_train)
min_len = min(len(y_pred), len(y_train))
dates_array = df_vendas['Data do pedido'].values 

for i in range(min_len):
    data_pedido = dates_array[i]
    real = df_vendas['Vendas'].iloc[i] 
    previsto = y_pred[i][0]
    print(f"Data: {data_pedido}, Real: {real}, Previsto: {previsto}")




66/66 [==============================] - 0s 6ms/step
Data: 2021-02-10T00:00:00.000000000, Real: 0.003173477153783274, Previsto: 0.6097321510314941
Data: 2021-03-10T00:00:00.000000000, Real: 0.017144058480301424, Previsto: -0.37371397018432617
Data: 2021-04-07T00:00:00.000000000, Real: 0.1547534036775691, Previsto: -0.8572726249694824
Data: 2021-04-07T00:00:00.000000000, Real: 0.09728551428651966, Previsto: -0.27394628524780273
Data: 2021-04-07T00:00:00.000000000, Real: 0.011876462242433924, Previsto: -0.6536593437194824
Data: 2021-09-12T00:00:00.000000000, Real: 0.017654962463238406, Previsto: 1.0069622993469238
Data: 2021-06-05T00:00:00.000000000, Real: 0.012516560335998649, Previsto: -0.9264941215515137
Data: 2018-03-04T00:00:00.000000000, Real: 0.06706364101889523, Previsto: -0.6391024589538574
Data: 2018-03-04T00:00:00.000000000, Real: 0.04996502363077733, Previsto: -0.7987551689147949
Data: 2018-03-04T00:00:00.000000000, Real: 0.20002302004153005, Previsto: 0.22927521169185638
Dat